In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import gradio as gr
import os

load_dotenv(override=True)
openai_api_key = os.getenv('GOOGLE_API_KEY')
openai_url = os.getenv('GEMINI_BASE_URL')
gemini_model = os.getenv('GEMINI_MODEL')

def get_openai_client():
    return OpenAI(base_url=openai_url, api_key=openai_api_key)

def send_message(client, message,role="user", model=gemini_model):
    return client.chat.completions.create(
        model=model,
        messages=[
            {"role": role, "content": message}
        ]
    )

def send_messages(client, messages, model=gemini_model):
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

In [ ]:
with open("me/my_description.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [ ]:
name = "André Vizinha"

In [ ]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [ ]:
system_prompt

In [ ]:
client = get_openai_client()

def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = send_messages(client, messages)
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

In [ ]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [ ]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [ ]:
evaluator_system_prompt

In [ ]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
evaluator = get_openai_client()

def evaluate(reply, message, history) -> Evaluation:
    messages=[{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = evaluator.beta.chat.completions.parse(model=gemini_model, messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [ ]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = send_messages(get_openai_client(), messages)
    return response.choices[0].message.content

In [ ]:
def chat1(message, history):
    if "sanji" in message:
        system = "You're Sanji the cook from Onepiece, you're the best cook there is, and you'll help anyone with your recipe knowledge"
        messages = [{"role": "system", "content": system}] + [{"role": "user", "content": message}]
    else:
        system = system_prompt
        messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = send_messages(get_openai_client(), messages)
    reply = response.choices[0].message.content
    evaluation = evaluate(reply, message, history)
    print(f"The evaluation passed? {evaluation.is_acceptable}")
    print(f"The evaluation feedback: {evaluation.feedback}")
    print(f"--------------------------------------------------")
    if ( not evaluation.is_acceptable):
        reply = rerun(reply, message, history, evaluation.feedback)
    
    return reply

In [ ]:
gr.ChatInterface(chat1, type="messages").launch()